In [1]:
import os
import numpy as np
import pandas as pd
import warnings

from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/drive")

os.chdir("drive/MyDrive/competition/2022-AI-competition-Round1")
os.listdir()

Mounted at /content/drive


['2022-08-06 Optimize ET.ipynb',
 'competition_data',
 'submission',
 'playground.ipynb',
 'AutoML Baseline.ipynb',
 'Evaluator Module.ipynb',
 '2022-08-04 feature EDA.ipynb',
 "2022-08-05 LGB_train(hyeonbin's parameter tuning).ipynb",
 'model compare.ipynb',
 'Optuna Optimization.ipynb',
 '2022-08-06 Regressor model compare.ipynb',
 'catboost_info',
 'evaluator',
 '.git',
 '.gitignore',
 'README.md',
 'Updated CLF.ipynb',
 '2022-08-17 Ensemble Module.ipynb',
 '2022-08-17 Evaluator Optuna (v0.4).ipynb',
 'GitHub Connection.ipynb',
 '2022-08-11 Models(XGB_ET) Optimization.ipynb']

In [2]:
!pip install -r evaluator/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 308 kB 59.4 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 209 kB 62.8 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 147 kB 62.6 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 112 kB 63.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ba6dab5dc140a869afa43b0566800dbc985bacd514e97bbf739d7a0c59aafe88
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
from evaluator.evaluator import Evaluator, Model

train_df = pd.read_csv('competition_data/train.csv')
test_df = pd.read_csv("competition_data/test.csv")
submission_df = pd.read_csv("competition_data/sample_submission.csv")

### baseline

In [6]:
Evaluator(
    **Model(train_df, "et", "clf").get_model()
).run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.780244,0.788391,0.831849,0.809537,0.872887,0.302898,NaN
2,0.784308,0.775799,0.849684,0.811062,0.877114,0.305249,NaN
3,0.764927,0.766200,0.830804,0.797195,0.861757,0.314742,NaN
4,0.784308,0.802514,0.830352,0.816196,0.867259,0.305771,NaN
mean,0.778446,0.783226,0.835672,0.808498,0.869754,0.307165,1.0


### optimize

In [4]:
model = Model(train_df, "et", "clf")

In [13]:
initial_params = (
    ("n_estimators", "int", (200, 500)),
    ("max_depth", "int", (10, 70)),
    ("min_samples_split", "int", (2, 5)),
    ("min_samples_leaf", "static", 1),
    # ("min_weight_fraction_leaf", "float", (0.0, 0.2)),
    ("max_features", "float", (0.6, 0.8)),
    ("max_leaf_nodes", "int", (300, 10000)),
    # ("min_impurity_decrease", "float", (0.0, 0.2)),
)
model.optimize(initial_params, n_runs=2, n_trials=100)

[I 2022-08-20 16:10:12,993] A new study created in memory with name: no-name-a94e107f-681c-4f4f-8ded-a1fa0772ff9e
[I 2022-08-20 16:11:52,344] Trial 0 finished with value: 0.8546059115077173 and parameters: {'n_estimators': 406, 'max_depth': 41, 'min_samples_split': 4, 'max_features': 0.6207417698896778, 'max_leaf_nodes': 771}. Best is trial 0 with value: 0.8546059115077173.
[I 2022-08-20 16:14:31,588] Trial 1 finished with value: 0.8712891949884845 and parameters: {'n_estimators': 410, 'max_depth': 36, 'min_samples_split': 2, 'max_features': 0.7675203838527384, 'max_leaf_nodes': 4526}. Best is trial 1 with value: 0.8712891949884845.
[I 2022-08-20 16:16:27,352] Trial 2 finished with value: 0.8550705081080487 and parameters: {'n_estimators': 479, 'max_depth': 36, 'min_samples_split': 3, 'max_features': 0.6153731452078782, 'max_leaf_nodes': 777}. Best is trial 1 with value: 0.8712891949884845.
[I 2022-08-20 16:18:07,989] Trial 3 finished with value: 0.8541836149203 and parameters: {'n_est

0.8728218850380565
{'n_estimators': 354, 'max_depth': 49, 'min_samples_split': 3, 'max_features': 0.6145048163782822, 'max_leaf_nodes': 5347}


In [4]:
params = {
    'n_estimators': 354, 
    'max_depth': 49, 
    'min_samples_split': 3, 
    'max_features': 0.6145048163782822, 
    'max_leaf_nodes': 5347    
}
evaluator = Evaluator(
    **Model(train_df, "et", "clf", **params).get_model()
)
evaluator.run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.782745,0.791733,0.831849,0.811295,0.872830,0.296283,NaN
2,0.789622,0.783069,0.849111,0.814754,0.878051,0.298783,NaN
3,0.782119,0.783246,0.840922,0.811060,0.867789,0.305430,NaN
4,0.785245,0.805380,0.827642,0.816359,0.869392,0.299748,NaN
mean,0.784933,0.790857,0.837381,0.813367,0.872016,0.300061,1.0


In [6]:
params = {
    'n_estimators': 354, 
    'max_depth': 49, 
    'min_samples_split': 3, 
    'max_features': 0.6145048163782822, 
    'max_leaf_nodes': 5347    
}
evaluator = Evaluator(
    **Model(train_df, "et", "clf", **params).get_model()
)
evaluator.train_best_model(n_runs=10)

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.783682,0.792063,0.833519,0.812263,0.872230,0.296656,NaN
2,0.790247,0.785714,0.845668,0.814590,0.878691,0.298331,NaN
3,0.782745,0.782881,0.843170,0.811908,0.870692,0.304198,NaN
4,0.786808,0.807184,0.828184,0.817549,0.870603,0.298462,NaN
mean,0.785871,0.791961,0.837635,0.814077,0.873054,0.299412,1.0
